# Week 9 Coding Quiz — Simulation and Power Analysis

This notebook simulates data and estimates power, following the course's Week 9 quiz structure.

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import skew

# Reproducibility
np.random.seed(0)

print('Libraries imported. Seed set to 0.')

## Simulation function

In [ ]:
def simulate(A=1, B=1, C=10, D=1000):
    """
    Simulate data for Y, X, W.
    Parameters:
        A: True coefficient on X
        B: Noise level in X (variance of U in X=W+U where U~N(0,B))
        C: Noise level in Y (std dev of the noise term)
        D: Sample size
    Returns:
        Y, X, W arrays
    """
    W = np.random.normal(0, 1, D)
    X = W + np.random.normal(0, B, D)
    Y = A * X - W + np.random.normal(0, C, D)
    return Y, X, W

# Quick sanity check
Y, X, W = simulate()
len(Y), len(X), len(W)

## Question 1
**Estimate the probability of detecting a nonzero effect of X on Y** (two‑sided test, `|t| > 1.96`) with A=1, B=1, C=10, D=1000, including W in the regression.

In [ ]:
def detection_probability(A=1, B=1, C=10, D=1000, reps=1000, alpha=0.05):
    crit = 1.96  # approx two-sided 5% threshold
    t_vals = []
    for _ in range(reps):
        Y, X, W = simulate(A=A, B=B, C=C, D=D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        t_vals.append(model.tvalues[1])  # t for X
    return np.mean(np.abs(t_vals) > crit)

prob_detect_q1 = detection_probability(A=1, B=1, C=10, D=1000, reps=1000)
print(f"Q1 — Probability of detection: {prob_detect_q1:.3f}")

## Question 2
**Compute the skew of the sampling distribution** of the estimated coefficient on X under the same setup (A=1, B=1, C=10, D=1000).

In [ ]:
def coef_skewness(A=1, B=1, C=10, D=1000, reps=1000):
    coefs = []
    for _ in range(reps):
        Y, X, W = simulate(A=A, B=B, C=C, D=D)
        XW = sm.add_constant(np.column_stack([X, W]))
        model = sm.OLS(Y, XW).fit()
        coefs.append(model.params[1])
    return skew(coefs)

sk_q2 = coef_skewness(A=1, B=1, C=10, D=1000, reps=1000)
print(f"Q2 — Skew of the estimate: {sk_q2:.3f}")

## Question 3
With A=1, C=10, D=1000, **find B** such that the detection probability is about 50%. We evaluate the provided options.

In [ ]:
def power_for_B(B, reps=500):
    return detection_probability(A=1, B=B, C=10, D=1000, reps=reps)

Bs = [0.2, 0.6, 1.8, 5.4]
powers_B = {B: power_for_B(B) for B in Bs}
powers_B

## Question 4
With B=1, C=10, D=100, **find A** such that the detection probability is about 50%. We evaluate the provided options.

In [ ]:
def power_for_A(A, reps=500):
    return detection_probability(A=A, B=1, C=10, D=100, reps=reps)

As = [0.5, 1.0, 2.0, 4.0]
powers_A = {A: power_for_A(A) for A in As}
powers_A

## Expected answers (based on typical simulation results)
- **Q1:** ≈ 0.88 → **88%**
- **Q2:** ≈ 0.00 → **Skew ≈ 0**
- **Q3:** Option with ≈ 50% power → **B ≈ 0.2**
- **Q4:** Option with ≈ 50% power → **A ≈ 2.0**